In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from io import StringIO


## Fonctions auxiliaires pour trouver la topologie à partir du fichier PDB

In [ ]:
class guanine:
    def __init__(self,conformation,nb):
        self.conformation = conformation #anti = 1; syn = 0
        self.nb = nb

class G4:
    def __init__(self,tetrades,nb_mol,topo,bool):
        self.t = tetrades
        self.topologie = topo
        self.nb_mol = nb_mol
        self.diag_loop = bool

In [ ]:
cut_H1 = 3.075641804556364

def classication_syn_anti(G):    #Regarde la distance entre H8 et H1' qui est caractéristique de syn/anti
    H8 = G.loc[ (G[2] == 'H8') ,6:8].values[0]            #On prend la position de H8
    H1p = G.loc[ (G[2] == 'H1\'') ,6:8].values[0]
    dist = np.linalg.norm(H8-H1p)
    if dist>cut_H1:
        return 1  #anti
    else:
        return 0 #sin

def g_from_nb(g_list,nb):   #obtenir la class guanine à partir du nombre de base
    i = 0
    while g_list[i].nb != nb:
        i+=1
    return g_list[i]

def build_g4(name):

    text = urllib.request.urlopen('https://files.rcsb.org/view/'+name+'.pdb').read().decode()

    n_mol = 0

    def following_line(text,k):
        c = k
        while text[c] != '\n':
            c+=1
        return c+1

    ind_start = 0
    ind_end = 0

    while text[ind_start:ind_start+4]!= "ATOM":      #on tronque le début du fichier qui ne nous est pas utile pour l'instant
        ind_start = following_line(text,ind_start)

    ind_end = ind_start
    bool = True
    while bool:
        n_mol +=1
        bool = False
        while text[ind_end:ind_end+3] != "TER":   #idem pour la fin du fichier
            ind_end = following_line(text,ind_end)
        f_l = following_line(text,ind_end)
        if text[f_l:f_l+4] == "ATOM":
            ind_end = f_l
            bool = True

    file = StringIO(text[ind_start:ind_end])

    data = pd.read_csv (file, delimiter=" +", header = None,engine = 'python')


    nucleotid_number = []

    current_nb_base = 1
    last_nb_base = 1
    for k in range(len(data)):  #On résout le pb de repartir à la base n°1 si plusieurs molécules
        nb_base = data.iloc[k,5]
        if nb_base == current_nb_base + 1:
            current_nb_base += 1
            last_nb_base = nb_base
        elif nb_base < current_nb_base -1:
            if nb_base != last_nb_base:
                current_nb_base +=1
                last_nb_base = nb_base
                data.iloc[k,5] = current_nb_base
            else:
                data.iloc[k,5] = current_nb_base
    for k in range(len(data)):                 #on sélectionne les numéros de nucléotides correspondant à des G
        if data.iloc[k,3] == "DG":
            nb = data.iloc[k,5]
            if not nb in nucleotid_number:
                nucleotid_number.append(nb)
    anti = 0
    syn = 0

    guanine_list = []

    for nb in nucleotid_number:
        G = data.loc[ data[5]== nb,:]
        classe = classication_syn_anti(G)
        g = guanine(classe,nb)
        guanine_list.append(g)

    tetrades_list = []
    tetrades_g_list = []
    numbers = len(nucleotid_number) * [0] #On regarde les numéros de G qui sont déjà dans un tetrade
    ones = len(nucleotid_number)*[1]
    while numbers != ones:
        tetrade = []
        tetrade_g = []
        i = 0
        while numbers[i]==1: #on prend le premier numéro qui n'est pas déjà traité
            i+=1
        indexs = [i]
        O6 = data.loc[ (data[2] == 'O6') & (data[5] == nucleotid_number[i]) ,6:8].values[0]  #distance O6-H1 caractéristique du fait d'être côte à côte
        H1 = data.loc[ (data[2] == 'H1') & (data[5] == nucleotid_number[i]),6:8].values[0]
        tetrade.append(nucleotid_number[i])
        tetrade_g.append(g_from_nb(guanine_list,nucleotid_number[i]))
        bool = True
        while bool and len(tetrade)<4:
            bool = False  #regarde si on a ajouté un autre G à la tetrade en cours
            for k in range(len(nucleotid_number)):
                if (not nucleotid_number[k] in tetrade) and numbers[k]==0:
                    O6bis = data.loc[ (data[2] == 'O6') & (data[5] == nucleotid_number[k]) ,6:8].values[0]
                    H1bis = data.loc[ (data[2] == 'H1') & (data[5] == nucleotid_number[k]),6:8].values[0]
                    if np.linalg.norm(O6bis - H1) < 2.5 or np.linalg.norm(O6 - H1bis) < 2.5:
                        tetrade.append(nucleotid_number[k])
                        tetrade_g.append(g_from_nb(guanine_list,nucleotid_number[k]))
                        bool = True
                        indexs.append(k)
                        O6 = O6bis
                        H1 = H1bis
        for x in indexs:
            numbers[x] =  1
        if len(tetrade)>3:
            tetrades_list.append(tetrade)
            tetrades_g_list.append(tetrade_g)
    t1,t2 = tetrades_list[0],tetrades_list[1]
    up = 0
    down = 0
    diag_loop = False
    for i in range(len(t1)-1):
        if t1[i]>t1[i+1]:
            diag_loop = True
            continue
    for i in range(len(t1)):
        if t1[i]<t2[i]:
            up+=1
        else:
            down+=1
    if up == 0 or down == 0:
        ratio = 10  #parallel
    else:
        ratio = np.max([up/down,down/up])
    if ratio == 10:
        topo = "parallel"
    elif ratio == 1:
        topo = "antiparallel"
    else:
        topo = "hybrid"
    return G4(tetrades_g_list,n_mol,topo,diag_loop)

In [ ]:
def Topologie(name):  ## name = PDB id
    quad = build_g4(name)
    t_list = quad.t
    for i in range(len(t_list)):
        print("\n Tétrade "+str(i+1)+" : ")
        for g in t_list[i]:
            if g.conformation == 1:
                conf = "anti"
            else:
                conf = "syn"
            print(str(int(g.nb))+" ("+conf+")   ")
    nmol = quad.nb_mol
    if nmol == 1:
        type = "monomeric"
    elif nmol == 2:
        type = "dimeric"
    elif nmol == 3:
        type = "trimeric"
    else:
        type = "tetrameric"
    print("\n => "+quad.topologie+ "  ("+type+")")
    if quad.diag_loop and nmol ==1 :
        print("\n There is a diagonal loop")

## Exemple d'application de l'algo

In [ ]:
Topologie("2kpr")   ## donner en entrée l'id PDB du G4


 Tétrade 1 : 
1 (syn)   
6 (anti)   
17 (anti)   
13 (anti)   

 Tétrade 2 : 
2 (anti)   
5 (syn)   
16 (syn)   
12 (anti)   

 Tétrade 3 : 
7 (anti)   
11 (syn)   
14 (anti)   
18 (anti)   

 => hybrid  (monomeric)

 There is a diagonal loop
